importing and viewing the dataset

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
import string
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import statistics
from statistics import mode
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity


train_data = pd.read_csv("1598639150_466036_train_file.txt", sep="\t")
print(train_data)

test_data = pd.read_csv("1598639150_4984343_test_file.txt", header=None, names=["review"])
print(test_data)

In [ ]:
train_data['review'].count()

In [ ]:
f = open("1598639150_4984343_test_file.txt", 'r', encoding="utf8")
txt = f.read().split('\n')
txt = list(filter(None, txt))
test_data = pd.DataFrame(txt, columns=['review'])
test_data

check for null values in dataset

In [ ]:
train_data.isnull().sum()
test_data.isnull().sum()

data cleaning and preprocessing
1. removing \<br /> tag

download nltk

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

convert sentences to vectors

In [ ]:
stopwords = stopwords.words('english')


In [ ]:
stopwords.append('br')

In [ ]:
def clean_string(cleaned_text):
    lemmatizer = WordNetLemmatizer()
    #      remove tags
    cleaned_text = re.sub('<[^<]+?>','', cleaned_text)
    #     to lower case
    cleaned_text = cleaned_text.lower()   
    #     lemmatization
    cleaned_text = [lemmatizer.lemmatize(word) for word in cleaned_text]
    #     remove numbers
    cleaned_text = ''.join(word for word in cleaned_text if not word.isdigit())
    #      remove punctuation
    cleaned_text = ''.join([word for word in cleaned_text if word not in string.punctuation])
    #     remove stop-words
    cleaned_text = ' '.join([word for word in cleaned_text.split() if word not in stopwords])
    
    return cleaned_text

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
train_data['review'] = train_data['review'].str.lower()
train_data['review'] = train_data['review'].str.replace('[^a-zA-Z ]', ' ')
train_data['review'] = train_data['review'].apply(lambda x : ' '.join([lemmatizer.lemmatize(word) for word in x.split() if word not in stopwords and len(word) > 1]))

In [ ]:
test_data['review'] = test_data['review'].str.lower()
test_data['review'] = test_data['review'].str.replace('[^a-zA-Z ]', ' ')
test_data['review'] = test_data['review'].apply(lambda x : ' '.join([lemmatizer.lemmatize(word) for word in x.split() if word not in stopwords and len(word) > 1]))

In [ ]:
train_data['review'][:10]

In [ ]:
# test_data['review'][:10]

In [ ]:
#cleaned_train = list(map(clean_string, train_data['review']))
#cleaned_test = list(map(clean_string, test_data['review']))
#cleaned_train
#len(cleaned_test)

In [ ]:
train_vectorizer = TfidfVectorizer(min_df=2, ngram_range=(1,3), max_features = 75000)
train_vectorizer.fit(train_data['review'])
train_vector = train_vectorizer.transform(train_data['review'])

test_vector = train_vectorizer.transform(test_data['review'])

# train_vectorizer.fit(cleaned_test)
# test_vector = train_vectorizer.transform(cleaned_test)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_vector, train_data['sentiment'], test_size=0.2, random_state=42)
# print(y_train.shape, y_test.shape)
# X_train = train_vector
# X_test = test_vector
# X_train = train_vector[:train_data['review'].count()]
# X_test = train_vector[train_data['review'].count():]
print(train_vector.shape)
# X_test = train_vector[-len(cleaned_test):len(cleaned_train)]

# y_train = train_data['sentiment']
print(X_train.shape, X_test.shape, y_train)


In [ ]:
y_train = y_train.to_numpy()
#print(y_train)
print(y_train.shape)
print(type(y_train))

In [ ]:
y_test = y_test.to_numpy()
#print(y_test)
print(y_test.shape)
print(type(y_test))

In [ ]:
print(type(y_train))
print(type(y_test))

In [ ]:
cos_sim = cosine_similarity(X_train, X_test)
cos_sim = cos_sim.transpose()
print(type(cos_sim))
print(cos_sim.shape)
print(cos_sim)

In [ ]:
k = 3
for k in range(1,300,2):
    indices = list()
#     k = 181
    for doc in cos_sim:
        indices.append(np.argsort(doc)[-k:])
    pred_val = list()
    for arr in indices:
        pred_val.append(mode([y_train[i] for i in arr]))
    correct = 0
    for i in range(len(pred_val)):
        if pred_val[i] == y_test[i]:
            correct+=1
    print("For k = ",k," => ", correct/len(pred_val)*100, "%\n")

In [ ]:
indices = list()
k = 211
for doc in cos_sim:
    indices.append(np.argsort(doc)[-k:])

In [ ]:
indices[0]

In [ ]:
pred_val = list()
for arr in indices:
    pred_val.append(mode([y_train[i] for i in arr]))

In [ ]:
print(len(pred_val))

In [ ]:
print(pred_val)

In [ ]:
correct = 0

for i in range(len(pred_val)):
    if pred_val[i] == y_test[i]:
        correct+=1

In [ ]:
print(correct/len(pred_val)*100, "%")

In [ ]:
with open('pred_values.txt', 'w') as filehandle:
    for listitem in pred_val:
        filehandle.write('%s\n' % listitem)